In [1]:
import matplotlib.pyplot as plt
import pandas as p
import seaborn as sns
import numpy as np
%matplotlib inline
from scipy.stats import linregress
from scipy.spatial import distance
from matplotlib import colors
import scipy
import os
import copy
from matplotlib.backends.backend_pdf import PdfPages

from ast import literal_eval
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from itertools import combinations

import Levenshtein

sns.set_color_codes()
sns.set_style('white')
sns.set_style('ticks')




In [2]:
# previous_sources = p.read_csv('barcodes_previously_used.csv')


In [3]:
# big_pool = p.read_csv('curatedBClist.csv',index_col=0)
# big_pool['barcode'] = [f'{bc2}_{bc1}' for bc1,bc2 in zip(big_pool['BC1'].values,big_pool['BC2'].values)]

# bc_csv = p.read_csv('barcodes_previous+new.csv')
bc_csv = p.read_csv('SequenceToCount/barcodes_curated_30April2022.csv')

In [4]:
bc_csv

,BCID,barcode_fasta,barcode,barcode_lowcomplexity,barcode_highcomplexity,source_publication,cloneID,cloneID_new,evolution_condition,ancestor,...,1D_Error_Li2019,2D_Fitness_Li2019,2D_Error_Li2019,3D_Fitness_Li2019,3D_Error_Li2019,5D_Fitness_Li2019,5D_Error_Li2019,FerPerHour_Li2019,ResPerHour_Li2019,StaPerHour_Li2019
0,0,GAGTGAAACATGAATGGTATTCATCATGATCAATCTACAAAAATAT...,TGATCAATCTACAAAAATATTTAATG_GAGTGAAACATGAATGGTA...,TGATCAATCTACAAAAATATTTAATG,GAGTGAAACATGAATGGTATTCATCA,This_study,NaN,EVO2D_IRA1_MIS_rearray_Plate1-A1,Evo2D,IRA1_MIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ACTCTAACGTGTAACTAATTTTGAGTCCGCCAATCCCGAACCCCGT...,CCGCCAATCCCGAACCCCGTTTCGCC_ACTCTAACGTGTAACTAAT...,CCGCCAATCCCGAACCCCGTTTCGCC,ACTCTAACGTGTAACTAATTTTGAGT,Aggeli2020,gpb2_evo1_plate9_H8,gpb2_evo1_plate9_H8,Evo2D,GPB2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ATGGGAACACGGAATGATCTTTTATTGACAGAAAAGCCAAATGGAT...,GACAGAAAAGCCAAATGGATTTACCG_ATGGGAACACGGAATGATC...,GACAGAAAAGCCAAATGGATTTACCG,ATGGGAACACGGAATGATCTTTTATT,This_study,NaN,EVO3D_TCG_rearray_Plate1-A4,Evo3D,CYR1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,TACTAAATAAGTAAGGGAATTCTGTTCCAACAAAACACAAATCTGT...,CCAACAAAACACAAATCTGTTGTGTA_TACTAAATAAGTAAGGGAA...,CCAACAAAACACAAATCTGTTGTGTA,TACTAAATAAGTAAGGGAATTCTGTT,This_study,NaN,EVO2D_IRA1_NON_rearray_Plate1-A1,Evo2D,IRA1_NON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ATGACAAACTTTAAATAATTTAATTGTATCGAAACCCAAAGAGATT...,TATCGAAACCCAAAGAGATTTAATCG_ATGACAAACTTTAAATAAT...,TATCGAAACCCAAAGAGATTTAATCG,ATGACAAACTTTAAATAATTTAATTG,This_study,NaN,EVO3D_IRA1_MIS_rearray_Plate1-C4,Evo3D,IRA1_MIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5022,5022,GATCCGATATCGGTACCGTTTCAAAATTAAAAATACAAAAAAAGAT...,TTAAAAATACAAAAAAAGATTTAAGG_GATCCGATATCGGTACCGT...,TTAAAAATACAAAAAAAGATTTAAGG,GATCCGATATCGGTACCGTTTCAAAA,Aggeli2020,NaN,NaN,Evo2D,TOR1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5023,5023,TCCGATATCGGTACCATCGTTTTCTTTTAAAAATACAAAAAAAGAT...,TTAAAAATACAAAAAAAGATTTAAGG_TCCGATATCGGTACCATCG...,TTAAAAATACAAAAAAAGATTTAAGG,TCCGATATCGGTACCATCGTTTTCTT,Aggeli2020,NaN,NaN,Evo2D,TOR1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5024,5024,TCGGTACTACAAAAATGATTTGGGGCATCAGAAGTTCGAATCAAAT...,ATCAGAAGTTCGAATCAAATTACGAA_TCGGTACTACAAAAATGAT...,ATCAGAAGTTCGAATCAAATTACGAA,TCGGTACTACAAAAATGATTTGGGGC,Aggeli2020,NaN,NaN,Evo2D,TOR1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5025,5025,TCTACAAATCTAAGTATTGTACATTAAAAATACAAAAAAAGATTTAAGG,TTAAAAATACAAAAAAAGATTTAAGG_TCTACAAATCTAAGTATTG...,TTAAAAATACAAAAAAAGATTTAAGG,TCTACAAATCTAAGTATTGTACA,Aggeli2020,tor1_evo1_plate1_E12,tor1_evo1_plate1_E12,Evo2D,TOR1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# bc_counts = p.read_table('allBarcodeCounts_levenshteinRead1First_08Sept2020.tab')
# bc_sequences = SeqIO.parse(open('BarcodeCounter_scripts/bartender_levenshtein_BClist.fa'),'fasta')

# bc_counts = p.read_table('allBarcodeCounts_curatedBClist.tab')
# bc_counts = p.read_table('allBarcodeCounts_truncated16Dec2020.tab')

# bc_counts = p.read_table('allBarcodeCounts_BestList.tab')
bc_counts = p.read_table('data/2%5%_allBarcodeCounts.tab')
bc_sequences = SeqIO.parse(open('SequenceToCount/barcodes_curated_30April2022.fasta'),'fasta')

ids = []
seqs = []
id_seq_dict = {}
for x in bc_sequences:
    ids.append(x.id)
    seqs.append(x.seq)
    id_seq_dict[x.id] = x.seq

sequences = []
for bc_id in bc_counts['BCID'].values:
    sequences.append(str(id_seq_dict[str(bc_id)]))
    
bc_counts['bc_merged'] = sequences

bc_seq = []
for merged_bc in bc_counts['bc_merged']:
    this_bc = bc_csv[bc_csv['barcode_fasta']==merged_bc]
    if len(this_bc['barcode'].values) < 1:
        print(merged_bc)
        if merged_bc == 'CGCTAAAGACATAATGTGGTTTGTTGGAATTAAGTTGGAACAGTGTTGACAGGTGCAC':
            bc_seq.append('CGCTAAAGACATAATGTGGTTTGTTG_GAATTAAGTTGGAACAGTGTTGACAGGTGCAC')
#             print('ancestor sequence! - not adding' )
    else:
        bc_seq.append(this_bc['barcode'].values[0])

bc_counts['barcode'] = bc_seq
bc_counts['barcode_lowcomplexity'] = [bc.split('_')[0] for bc in bc_counts['barcode'].values]
bc_counts['barcode_highcomplexity'] = [bc.split('_')[1] for bc in bc_counts['barcode'].values]



CGCTAAAGACATAATGTGGTTTGTTGGAATTAAGTTGGAACAGTGTTGACAGGTGCAC


In [12]:
# bc_counts.to_csv('barcode_counts_07Jan2021_withsequence.csv',index=False)
bc_counts.to_csv('data/2%5%_FitnessMeasurement_barcode_counts_withsequence.csv',index=False)

# bc_counts.to_csv('barcode_counts_TRUNCATED_curated_16Dec2020_withsequence.csv',index=False)